<a href="https://colab.research.google.com/github/Yasminebenhamadi/NMA/blob/main/Python/KL/diag/merge_end/merging_strategy_repeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn import mixture
import random
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import scipy.stats
from sklearn.datasets import make_spd_matrix,make_blobs
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture as GMM
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal as mvn
from sklearn.cluster import KMeans
from sklearn.metrics import pair_confusion_matrix, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from numpy import linalg as la
from sklearn.preprocessing import MinMaxScaler
from scipy.stats.stats import kruskal
from sklearn.manifold import TSNE
from scipy.stats import zscore
import re
from sklearn.model_selection import GridSearchCV
import os
import csv


plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize']=14,6

/tmp/ipykernel_12317/3443411012.py:19: DeprecationWarning: Please use `kruskal` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import kruskal


### utilities

In [ ]:
#@title Figure Settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

In [ ]:
def visualize_components(component1, component2, labels, show=True):
  """
  Plots a 2D representation of the data for visualization with categories
  labelled as different colors.

  Args:
    component1 (numpy array of floats) : Vector of component 1 scores
    component2 (numpy array of floats) : Vector of component 2 scores
    labels (numpy array of floats)     : Vector corresponding to categories of
                                         samples

  Returns:
    Nothing.

  """

  plt.figure()
  cmap = plt.cm.get_cmap('tab10')
  plt.scatter(x=component1, y=component2, c=labels, cmap=cmap)
  plt.xlabel('Component 1')
  plt.ylabel('Component 2')
  plt.colorbar(ticks=range(10))
  plt.clim(-0.5, 9.5)
  if show:
    plt.show()

In [ ]:
def ari(labels_true,labels_pred): 
    '''safer implementation of ari score calculation'''
    (tn, fp), (fn, tp) = pair_confusion_matrix(labels_true, labels_pred)
    tn=int(tn)
    tp=int(tp)
    fp=int(fp)
    fn=int(fn)

    # Special cases: empty data or full agreement
    if fn == 0 and fp == 0:
        return 1.0

    return 2. * (tp * tn - fn * fp) / ((tp + fn) * (fn + tn) +
                                       (tp + fp) * (fp + tn))

In [ ]:
def f1_score(labels_true,labels_pred): 
    '''safer implementation of ari score calculation'''
    (tn, fp), (fn, tp) = pair_confusion_matrix(labels_true, labels_pred)
    tn=int(tn)
    tp=int(tp)
    fp=int(fp)
    fn=int(fn)

    precision= tp/(tp+fp)
    recall= tp/(tp+fn)

    # Special cases: empty data or full agreement
    if fn == 0 and fp == 0:
        return 1.0

    return 2. * precision * recall / (precision+recall)

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
def min_max(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

## Ploting

In [ ]:
def plot_bivariate_data(X, title):
  fig = plt.figure(figsize=[8, 4])
  gs = fig.add_gridspec(2, 2)
  ax = fig.add_subplot(gs[:, 1])
  ax.plot(X[:, 0], X[:, 1], '.', markerfacecolor=[.5, .5, .5],
           markeredgewidth=0)
  plt.xlabel('Feature 1')
  plt.ylabel('Feature 2')
  plt.title(title)
  plt.show()

In [ ]:
def plot_contours(data, means, covs,labels, title, xa=[-12,12],ya=[-12,12]):
    """visualize the gaussian components over the data"""
    plt.figure()
    cmap = plt.cm.get_cmap('tab10')
    plt.scatter(data[:, 0], data[:, 1],c=labels, cmap=cmap, s=40 ,alpha=0.4)
    

    delta = 0.025
    if len(means)>0:
      k = means.shape[0]
    else:
      k=0
    x = np.arange(xa[0], xa[1], delta)
    y = np.arange(ya[0], ya[1], delta)
    x_grid, y_grid = np.meshgrid(x, y)
    coordinates = np.array([x_grid.ravel(), y_grid.ravel()]).T

    col = ['cyan', 'red', 'indigo','blue','white']
    for i in range(k):
        mean = means[i]
        cov = covs[i]
        z_grid = multivariate_normal(mean, cov).pdf(coordinates).reshape(x_grid.shape)
        plt.contour(x_grid, y_grid, z_grid, colors = col[i])

    plt.title(title)
    plt.tight_layout()

# KL Divergence

In [ ]:
def kl_mvn(m0, S0, m1, S1):
    """
    Kullback-Liebler divergence from Gaussian pm,pv to Gaussian qm,qv.
    Also computes KL divergence from a single Gaussian pm,pv to a set
    of Gaussians qm,qv.
    

    From wikipedia
    KL( (m0, S0) || (m1, S1))
         = .5 * ( tr(S1^{-1} S0) + log |S1|/|S0| + 
                  (m1 - m0)^T S1^{-1} (m1 - m0) - N )
    """
    
    # store inv diag covariance of S1 and diff between means
    N = m0.shape[0]
    iS1 = np.linalg.inv(S1)
    #way to calculate the determinant to avoid underflow and overflow
    sign,logdetS0=np.linalg.slogdet(S0)
   # detS0=sign * np.exp(logdetS0)
    sign,logdetS1=np.linalg.slogdet(S1)
    #detS1=sign * np.exp(logdetS1)

    diff = m1 - m0
    # kl is made of three terms
    tr_term   = np.trace(iS1 @ S0)
    det_term  = logdetS1-logdetS0
   # print(det_term) #np.sum(np.log(S1)) - np.sum(np.log(S0))
    quad_term = diff.T @ np.linalg.inv(S1) @ diff #np.sum( (diff*diff) * iS1, axis=1)
    #print(tr_term,det_term,quad_term)
    return .5 * (tr_term + det_term + quad_term - N)

In [ ]:
def js_mvn(m0, s0, m1, s1):
  m2=(m0+m1)/2
  s2=s0+s1
  return (kl_mvn(m0, s0, m2, s2) + kl_mvn(m1, s1, m2, s2))/2

In [ ]:
def KL_matrix(m0,S0,m1,S1):
  k0=m0.shape[0]
  k1=m1.shape[0]
  M=np.zeros((k0,k1))
  for i in range(k0):
    for j in range(k1):
      M[i,j]=kl_mvn(m0[i],S0[i], m1[j], S1[j])
  return M

# Incremental EM for GMM

In [ ]:
def trainGMM(data_x,n_components,dim, random_state,covariance_type='full',max_iter = 300,tol=1e-08):
  gmm = GMM(n_components=n_components, covariance_type=covariance_type, random_state=random_state, max_iter=max_iter,tol=tol,verbose=0)
  gmm.fit(data_x)
  return gmm.weights_,gmm.means_,gmm.covariances_,gmm


In [ ]:
def merge_two(n0,w0,m0,s0,n1,w1,m1,s1):
    new_mean=(n0*w0*m0+n1*w1*m1)/(n0*w0+n1*w1)
    new_weight=w0+w1#(n0*w0+n1*w1)/(n0+n1)
    s1=(n0*w0*s0+n1*w1*s1)/(n0*w0+n1*w1)
    sw=n0*w0*np.outer(np.transpose(m0),m0)+n1*w1*np.outer(np.transpose(m1),m1)
    s2=sw/(n0*w0+n1*w1)
    sub3=np.outer(np.transpose(new_mean),new_mean)
    new_cov=s1+s2-sub3
    n=n0
    return new_weight,new_mean,new_cov,n


In [ ]:
def incGMM(data,dim, n_components, increments_number, random_state, covariance_type='full',max_iter = 100,tol=1e-03, true_lables=[], incPrint=True):
  size_increments=int(len(data)/increments_number)
  clus_increment_size=int(size_increments/n_components)
  assignments=[]
  increments=[]
  weights=[]
  means=[]
  inc_labels=[]
  covariances=[]
  n0=0
  gmm = 0
  all_covs=[]
  all_means=[]
  all_weights=[]
  all_n=[]
  for i in range(increments_number):
      s=[i for j in range(size_increments)]
      inc_labels.append(s)
      if i == (increments_number - 1):
        inc = data[i*size_increments:data.shape[0],:]
      else:
        inc = data[i*size_increments:size_increments*(i+1),:]
      increments.append(inc)
      w,m,covs,gmm = trainGMM(inc,n_components,dim, covariance_type='full',random_state=random_state,max_iter = 500,tol=1e-06)
      
      all_means.append(m)
      all_covs.append(covs)
      all_weights.append(w)
      e=len(data)*np.ones((w.shape[0]))
      all_n.append(e)
      n1=len(inc)
      n0=n0+n1
      if incPrint and (len(true_lables)>0) and data.shape[1]==2:
        plot_contours(inc,means,covariances,true_lables[i*size_increments:size_increments*(i+1)], "increments_true")
        plt.savefig("inc_true{}.png".format(i))
  if incPrint and data.shape[1]==2:
    increments, inc_labels = np.array(increments).reshape(data.shape[0],dim) , np.array(inc_labels).reshape(data.shape[0])
    plot_contours(increments,[],[],inc_labels, "increments_partition")
  all_weights=np.array(all_weights)
  all_means=np.array(all_means)
  all_covs=np.array(all_covs)
  all_n=np.array(all_n)
  all_weights=all_weights.reshape(all_weights.shape[0]*all_weights.shape[1])/increments_number
  all_n=all_n.reshape(all_n.shape[0]*all_n.shape[1])
  all_means=all_means.reshape(all_means.shape[0]*all_means.shape[1],all_means.shape[2])
  all_covs=all_covs.reshape(all_covs.shape[0]*all_covs.shape[1],all_covs.shape[2],all_covs.shape[3])
  #all_n=np.array([int(j) for j in all_n])

  return all_weights,all_means,all_covs,all_n

# Tests

In [ ]:
def getFinalGmm(data, means, covariances, weights, true_labels = [], plot=True):
    gmm=GMM(n_components=weights.shape[0],covariance_type='full',max_iter=1)
    gmm.means_=means
    gmm.covariances_=covariances
    gmm.weights_=weights
    precisions_cholesky = np.linalg.inv(la.cholesky(covariances))
    gmm.precisions_cholesky_= np.array([np.transpose(p) for p in precisions_cholesky])
    assign = gmm.predict(data)
    if plot and len(true_labels>0) and data.shape[1]==2:
        plot_contours(data,means,covariances,assign, "increments")
    return gmm, assign


## R generated data

In [ ]:
def read_info_file(path):
    words = ["sepVal", "Number of clusters", "Number of dimensions", "Number of data points", "Number of outliers"]
    N, dim, number_comp, sepVal, outliers = (0,0,0,0,0)
    with open(path, 'r') as fp:
        # read all lines using readline()
        lines = fp.readlines()
        for row in lines:
            for word in words:
                if row.find(word) != -1:
                    x = row.split(' ')[-1]
                    if word == words[0]:
                        sepVal=float(x)
                    elif word == words[1]:
                        number_comp=int(x)
                    elif word == words[2]:
                        dim=int(x)
                    elif word == words[3]:
                        N=int(x)
                    else:
                        outliers=float(x)
                        
    return N, dim, number_comp, sepVal, outliers

In [ ]:
mainDir="/home/meriem/Documents/data/component/"
i=0
import time
time_eco=0
start_time = time.time()
with open(mainDir+'results_merging.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    header=["DatasetNumber", "size", "dim", "Percentage of increment", "cluster size", "number of clusters", "sepVal", "Percentage of outliers", "ari","min","max"]
    writer.writerow(header)
   
    for s in ['1','2','3','4']:
            data_file_chemin = mainDir+s+'/'
            print(data_file_chemin)
            files =os.listdir(data_file_chemin)
            if(len(files)>0):
                data_file =  data_file_chemin+[f for f in files if f.endswith(".mat")][0]
                print(data_file)
                labels_file =data_file_chemin+ [f for f in files if f.endswith(".mem")][0]
                info_file = data_file_chemin+[f for f in files if (f.endswith(".log") and not f.endswith("info.log"))][0]
                N, dim, number_comp, sepVal, outliers = read_info_file(info_file)

                data=pd.read_csv(data_file,sep=" ",header=None)
                data_labels=pd.read_csv(labels_file,header=None)
                data_labels.rename(columns = {0:2}, inplace = True)
                toute=pd.concat([data,data_labels],axis=1) 
                # sctt_plt = sns.scatterplot(data=toute, x=0, y=1, hue=2)
                # fig = sctt_plt.get_figure()
                # fig.savefig(dataset_dir+"out.png")
                # plt.clf()
                i=i+1
                toute=pd.concat([data,data_labels],axis=1)
                toute=toute.dropna()
                toute=toute.to_numpy()
                np.unique(data_labels.to_numpy(),return_counts=True)
                labels_true=data_labels.to_numpy().reshape(-1)
                data=data.to_numpy()
                data=min_max(data)
                #***********************************************GMM***********************************************
                trials=5
                for i in range(2,19):
                  print(i)
                  metrics=0
                  all_gmm=GMM(n_components=number_comp, covariance_type='full',max_iter = 300,tol=10e-8)
                  all_gmm.fit(data)
                  assign_all = all_gmm.predict(data)
                  metrics = np.zeros(len(header)-8)
                  minim=1
                  maxim=0
                  ari_all=0
                  debut=time.time()
                  for k in range(trials):
                    all_weights,all_means,all_covs,all_n=incGMM(data,dim, number_comp, i, k, 
                                                                covariance_type='full',max_iter = 300,tol=1e-08, 
                                                                true_lables=labels_true, incPrint=False)  
                    while (all_weights.shape[0]!=number_comp):
                            #print(all_means)
                            B=calculerM(all_weights,all_means,all_covs,all_n)
                            #print(B)
                            all_weights,all_means,all_covs,all_n=merge_operation(all_weights,all_means,all_covs,all_n,B)
                          #  print('sum ',sum(all_weights))
                            gmm, assign=getFinalGmm(data, all_means, all_covs, all_weights,labels_true , plot=False)
                    ari_one=ari(assign,labels_true)
                    ari_all=ari_one+ari_all
                    if (ari_one<minim):
                            minim=ari_one
                    if (ari_one>maxim):
                            maxim=ari_one
                  
                    print('max ',maxim,'min ',minim," ari ",ari_one)
                    print('sum ',sum(all_weights))
                  fin=time.time()-debut
                  time_eco=fin
                            

                        
                  metric = ari_all/trials
                  #print(metric)
                  info = [data_file, N, dim,i, int(N/number_comp), number_comp, sepVal, outliers]
                  info.extend([metric])
                  info.extend([minim])
                  info.extend([maxim])
                  info.extend([time_eco])
                  writer.writerow(info)
                  print('time  =======',time_eco)
                  time_eco=0
                 # print('number of inc ',i, '  ari  ',ari_all/trials)
                      #  print(all_means,all_n)
                    
                #calculerM(all_weights,all_means,all_covs,all_n)
    file.close()
print("--- %s seconds ---" % (time.time() - start_time))

/home/meriem/Documents/data/component/1/
/home/meriem/Documents/data/component/1/big-16000-0-30-2.mat
2
max  0.9660411430646106 min  0.9660411430646106  ari  0.9660411430646106
sum  1.0
max  0.9660411430646106 min  0.9660411430646106  ari  0.9660411430646106
sum  1.0
max  0.9660411430646106 min  0.9660411430646106  ari  0.9660411430646106
sum  1.0
max  0.9660411430646106 min  0.9660411430646106  ari  0.9660411430646106
sum  1.0
max  0.9660411430646106 min  0.9660411430646106  ari  0.9660411430646106
sum  1.0
time  ======= 2.8940303325653076
3
max  0.9660411430654066 min  0.9660411430654066  ari  0.9660411430654066
sum  1.0
max  0.9660411430654066 min  0.9660411430654066  ari  0.9660411430654066
sum  1.0
max  0.9660411430654066 min  0.9660411430654066  ari  0.9660411430654066
sum  0.9999999999999999
max  0.9660411430654066 min  0.9660411430654066  ari  0.9660411430654066
sum  1.0
max  0.9660411430654066 min  0.9660411430654066  ari  0.9660411430654066
sum  1.0
time  ======= 2.8904380798

In [ ]:
all_weights
sum(all_weights)

1.0

In [ ]:
all_gmm=GMM(n_components=number_comp, covariance_type='full',max_iter = 500,tol=10e-8,verbose=2)
all_gmm.fit(data)
assign_all = all_gmm.predict(data)

Initialization 0
Initialization converged: True	 time lapse 14.54879s	 ll 29.20863


In [ ]:
ari(assign_all,labels_true)

0.805213430286735

In [ ]:
ari(assign,labels_true)

0.9660411430677946

In [ ]:
def calculerM(all_weights,all_means,all_covs,all_n):
    n=all_weights.shape[0]
    A=np.zeros((n,n))
    for i in range(n):
        
        for j in range(n):
            if i<j:
                  m0=all_means[i]
                  m1=all_means[j]
                  S0=all_covs[i]
                  S1=all_covs[j]
                  A[i,j]=kl_mvn(m0, S0, m1, S1)
            else:
                A[i,j]=float('inf')
    return A


In [ ]:
def merge_operation(all_weights,all_means,all_covs,all_n,A):
    #print(np.argmin(B))
    ind=np.argwhere(B == np.min(B))[0]
    #get the indexes of min value
    
    i=ind[0]
    j=ind[1]
    m0=all_means[i]
    m1=all_means[j]
    S0=all_covs[i]
    S1=all_covs[j]
    w0=all_weights[i]
    w1=all_weights[j]
    n0=all_n[i]
    n1=all_n[j]
    #print('all_n',sum(all_n*all_weights))
    #delete the merged components
    if i<j:
        k=j
        j=i
        i=k
    A=np.delete(A,i,axis=0)
    A=np.delete(A,j,axis=0)
    A=np.delete(A,i,axis=1)
    A=np.delete(A,j,axis=1)
    all_covs=np.delete(all_covs,i,axis=0)
    all_covs=np.delete(all_covs,j,axis=0)
    all_means=np.delete(all_means,i,axis=0)
    all_means=np.delete(all_means,j,axis=0)

    all_weights=np.delete(all_weights,i,axis=0)
    all_weights=np.delete(all_weights,j,axis=0)

    all_n=np.delete(all_n,i,axis=0)
    all_n=np.delete(all_n,j,axis=0)
    

        
    
    
        
    new_weight,new_mean,new_cov,n=merge_two(n0,w0,m0,S0,n1,w1,m1,S1)  
    #n=n0+n1
#     all_weight=all_weights*all_n /sum(all_n*all_weights)
#     print('sum weights',sum(all_weight))
    

    all_means=np.append(all_means, [new_mean],axis=0)
    all_covs=np.append(all_covs,[new_cov],axis=0)
    all_weights=np.append(all_weights,[new_weight],axis=0)
    all_n=np.append(all_n,[n],axis=0)
  #  print(all_weights)
    return all_weights,all_means,all_covs,all_n
all_weights,all_means,all_covs,all_n=merge_operation(all_weights,all_means,all_covs,all_n,B)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
np.append(all_means,[[2,2]],axis=0)
all_covs.shape

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 30 and the array at index 1 has size 2

In [ ]:
np.argwhere(B == np.min(B))[0][0]

In [ ]:
sum(all_weights)